In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/공모전/NSMC-distilkobert

/content/drive/MyDrive/Colab Notebooks/공모전/NSMC-distilkobert


In [ ]:
# !git clone https://github.com/monologg/KoBERT-nsmc.git
%cd KoBERT-nsmc

/content/drive/MyDrive/Colab Notebooks/공모전/NSMC-distilkobert/KoBERT-nsmc


In [ ]:
!pip3 -q install transformers sentencepiece

     |████████████████████████████████| 2.8 MB 10.0 MB/s 
     |████████████████████████████████| 1.2 MB 72.1 MB/s 
     |████████████████████████████████| 52 kB 2.0 MB/s 
     |████████████████████████████████| 895 kB 78.4 MB/s 
     |████████████████████████████████| 3.3 MB 53.7 MB/s 
     |████████████████████████████████| 636 kB 48.1 MB/s 


In [ ]:
## TPU
#import os
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

#!pip install -q cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

# VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version $VERSION

In [ ]:
import pandas as pd
from pathlib import Path

def read_nsmc_split(split_dir):
    dataset = pd.read_csv(split_dir, sep='\t').loc[:, ['document', 'label']]
    dataset = dataset.loc[dataset["document"].isna().apply(lambda elm: not elm), :]
    texts = dataset['document'].tolist()
    labels = dataset['label'].tolist()
    
    return texts, labels

In [ ]:
train_texts, train_labels = read_nsmc_split('data/ratings_train.txt')
test_texts, test_labels = read_nsmc_split('data/ratings_test.txt')

In [ ]:
train_labels[:5]

[0, 1, 0, 0, 1]

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:
from tokenization_kobert import KoBertTokenizer

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
# imports pytorch
import torch

# imports the torch_xla package
# import torch_xla
# import torch_xla.core.xla_model as xm

class NSMCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = NSMCDataset(train_encodings, train_labels)
val_dataset  = NSMCDataset(val_encodings, val_labels)
test_dataset = NSMCDataset(test_encodings, test_labels)

In [ ]:
train_dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([   2, 4273, 5384, 3298, 7277, 1458, 3341, 6224, 7837, 3121, 5439, 4457,
         7088, 3824, 5760, 5496,    3,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1

In [ ]:
import time
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AdamW
from tqdm.auto import tqdm
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# Only TPU
# dev = xm.xla_device()

model = AutoModelForSequenceClassification.from_pretrained('monologg/distilkobert')
model.to(dev)
model.train()

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
optim = AdamW(model.parameters(), lr=5e-5)

for epoch in tqdm(range(5), desc='epochs'):
    # Perform training
    train_loss = 0.0
    start = time.time()
    for step, batch in tqdm(enumerate(train_loader), desc='steps', total=len(train_loader)):
        input_ids = batch['input_ids'].to(dev)
        attention_mask = batch['attention_mask'].to(dev)
        labels = batch['labels'].to(dev)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        train_loss += loss.item()
        optim.zero_grad()
        loss.backward()
        optim.step()
    print("iter %r: train loss/sent=%.4f, time=%.2fs" % (
            epoch, train_loss/len(train_loader), time.time()-start))

model.eval()

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/distilkobert were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.wei

epochs:   0%|          | 0/5 [00:00<?, ?it/s]

steps:   0%|          | 0/3750 [00:00<?, ?it/s]

...

steps:   0%|          | 0/3750 [00:00<?, ?it/s]

iter 3: train loss/sent=0.1693, time=790.98s


steps:   0%|          | 0/3750 [00:00<?, ?it/s]

iter 4: train loss/sent=0.1358, time=791.54s


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
        

In [ ]:
!ls

data		model			  README.md		  trainer.py
data_loader.py	model_			  requirements.txt	  utils.py
LICENSE		predict.py		  results
logs		__pycache__		  sample_pred_in.txt
main.py		pytorch-xla-env-setup.py  tokenization_kobert.py


In [ ]:
torch.save(model.state_dict(), 'model_/pytorch_model.bin')

In [ ]:
from transformers import AutoModelForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = AutoModelForSequenceClassification.from_pretrained('./model_')

## Inference

In [ ]:
s = ["이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.", 
     "아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.",
     "보면서 화가날수있습니다.",
     "아니 그래서 무슨말이 하고싶은거야 ㅋㅋㅋ",
     "아침드라마 김치싸대기보다 못한 액션과 출생의 비밀..",
     "취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지",
     "굳 ㅋ",
     "뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아"]
# 1 1 0 0 0 0 1 0


batch = tokenizer(s, truncation=True, padding=True)
input_ids = torch.tensor(batch['input_ids'])
attention_mask = torch.tensor(batch['attention_mask'])

output = model(input_ids, attention_mask=attention_mask)
logits = output.logits

In [ ]:
import torch.nn.functional as F
import numpy as np

torch.argmax(F.softmax(logits, dim=1), dim=1).numpy()
#F.softmax(logits, dim=1)

array([1, 1, 0, 0, 0, 0, 1, 0])